In [1]:
# Import the model code and all the libraries bcs they are so annoying otherwise
from LSTM import LyricsTokenizer, LyricsGenreDataset, LSTMLyrics_by_Genre, collate_fn, generate_lyrics, train_model
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import trange
import string

In [ ]:
import re
import os

id_pattern = re.compile(r'^[a-zA-Z0-9]{22},')

current_song_lyrics = []
first_id = False
first_line = True

all_songs = []
all_genres = []

folder_path = 'C:/Users/Chloe/Documents/Spring 2025/CSCI3832_FinalProject-main/LSTM_data'

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    genre = re.sub(r'\d+', '', os.path.splitext(filename)[0])
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            if first_line == True:
                first_line = False
                continue
            else:
                if id_pattern.match(line):
                    if first_id == False:
                        first_id = True
                    else:
                        lyrics_in_song = ''.join(current_song_lyrics).strip()
                        if len(lyrics_in_song) >= 10:
                            all_songs.append(lyrics_in_song)
                            all_genres.append(genre) 
                        current_song_lyrics = []
                else:
                    current_song_lyrics.append(line)  

        if current_song_lyrics:
            lyrics_in_song = ''.join(current_song_lyrics).strip()
            if len(lyrics_in_song) >= 10:
                all_songs.append(lyrics_in_song)   
                all_genres.append(genre)  
#remove punctuation
all_songs = [song.translate(str.maketrans('', '', string.punctuation)) for song in all_songs]
# #regular expression from geeks for geeks for removing non-eng char: https://www.geeksforgeeks.org/python-remove-non-english-characters-strings-from-list/
# all_songs = list(filter(lambda ele: re.search("[a-zA-Z]+", ele) is not None, all_songs))
# Build tokenizer
tokenizer = LyricsTokenizer(min_freq=1, max_length=100)#max length is 100 tokens for computing efficiency
tokenizer.build_vocab(all_songs)
tokenized_lyrics = tokenizer.tokenize(all_songs)

# Genre -> index mapping
genre2idx = {genre: idx for idx, genre in enumerate(sorted(set(all_genres)))}

# Dataset + Dataloader
dataset = LyricsGenreDataset(tokenized_lyrics, all_genres, genre2idx)

In [3]:
import re
import os

id_pattern = re.compile(r'^[a-zA-Z0-9]{22},')

first_id = False
first_line = True
folder_path = 'C:/Users/Chloe/Documents/Spring 2025/CSCI3832_FinalProject-main/LSTM_data'

all_songs = []
all_genres = []

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    genre = re.sub(r'\d+', '', os.path.splitext(filename)[0])

    with open(file_path, 'r', encoding='utf-8') as file:
        first_line = True
        current_song_lyrics = []
        for line in file:
            if first_line:
                first_line = False
                continue

            if id_pattern.match(line):
                if current_song_lyrics:
                    full_lyrics = ''.join(current_song_lyrics)
                    # remove punctuation, keep newlines
                    cleaned_lyrics = full_lyrics.translate(str.maketrans('', '', string.punctuation))
                    tokens = cleaned_lyrics.split()
                    # add the first and last 100 words as seperate entries to double data set size while maintaining computable sequence length
                    if len(tokens) >= 10:
                        if len(tokens) >= 100:
                            first_100 = ' '.join(tokens[:100])
                            last_100 = ' '.join(tokens[-100:])
                            all_songs.append(first_100)
                            all_genres.append(genre)
                            all_songs.append(last_100)
                            all_genres.append(genre)
                        else:
                            all_songs.append(' '.join(tokens))
                            all_genres.append(genre)

                    current_song_lyrics = []  # reset for next song
            else:
                current_song_lyrics.append(line)

        # Process last song in file
        if current_song_lyrics:
            full_lyrics = ''.join(current_song_lyrics)
            cleaned_lyrics = full_lyrics.translate(str.maketrans('', '', string.punctuation))
            tokens = cleaned_lyrics.split()
            if len(tokens) >= 10:
                if len(tokens) >= 100:
                    first_100 = ' '.join(tokens[:100])
                    last_100 = ' '.join(tokens[-100:])
                    all_songs.append(first_100)
                    all_genres.append(genre)
                    all_songs.append(last_100)
                    all_genres.append(genre)
                else:
                    all_songs.append(' '.join(tokens))
                    all_genres.append(genre) 

# #regular expression from geeks for geeks for removing non-eng char: https://www.geeksforgeeks.org/python-remove-non-english-characters-strings-from-list/
# all_songs = list(filter(lambda ele: re.search("[a-zA-Z]+", ele) is not None, all_songs))

In [5]:
# Build tokenizer
tokenizer = LyricsTokenizer(min_freq=1, max_length=100)#max length is 100 tokens for computing efficiency
tokenizer.build_vocab(all_songs)
tokenized_lyrics = tokenizer.tokenize(all_songs)

# Genre -> index mapping
genre2idx = {genre: idx for idx, genre in enumerate(sorted(set(all_genres)))}

# Dataset + Dataloader
dataset = LyricsGenreDataset(tokenized_lyrics, all_genres, genre2idx)

Missed 2721 out of 964450 words -- 0.28%


In [17]:
# Re-create the model
vocab_size = tokenizer.vocab_size()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_genres = len(genre2idx)


model = LSTMLyrics_by_Genre(
    vocab_size=vocab_size,
    embed_dim=256,
    hidden_size=256,
    genre_embed_size=32,
    num_layers=2,
    bidirectional=False,
    teacher_forcing_ratio=0.5,
    num_genres=num_genres
).to(device)

# Load saved weights
model.load_state_dict(torch.load('C:/Users/Chloe/Documents/Spring 2025/CSCI3832_FinalProject-main/src/model/LSTM_final.pth',weights_only=True))
model.eval()  # Put it in eval mode if you're generating or evaluating

lyrics = generate_lyrics(model, tokenizer, "rock", genre2idx=genre2idx, max_len=50, device=device, temperature=1)
print(lyrics)

chain stop oh sweet when the brothers is alright oh uh well get away together baby love hey hey hey what your love yeah now good well i get about the end its the got to stop to way to be know we is it you want love you feel


In [ ]:
import csv
filename = "rb_generated.csv"
lyrics = []

for i in range(100):
    lyric = generate_lyrics(model, tokenizer, "rb", genre2idx=genre2idx, max_len=100, device=device)
    if lyric.isascii():
        lyrics.append(lyric)

with open(filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows([[lyric] for lyric in lyrics])

print(f'CSV file "{filename}" created successfully.')
